In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import logsumexp
import random
import math
from scipy.stats import multivariate_normal
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.cluster import KMeans

In [2]:
train = np.array(pd.read_csv('./codon_usage.csv'))
# shuffle the dataset
train = np.delete(train, (486,5063), axis=0)
y = np.copy(train[:,0])
X = np.delete(train,(0,1,2,3,4), axis=1)
X = X.astype(np.float32)
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

c:\Users\Micha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


5

In [3]:
def gaussian_kernel(x, y, sigma=1):
    return np.exp(-((x - y) ** 2) / (2 * (sigma ** 2)))

def kmeans(X, k, max_iterations=100, sigma=1):
    # Choose the cluster centers from the dataset
    centers = X[np.random.choice(X.shape[0], size=k, replace=False)]

    for _ in range(max_iterations):
        # Calculate the distances between each point and the cluster centers
        distances = np.sqrt(((X[:, np.newaxis, :] - centers[np.newaxis, :, :]) ** 2).sum(axis=2))

        # Calculate the Gaussian kernel values
        kernel_values = gaussian_kernel(distances, 0, sigma)

        # Assign each point to the closest cluster center
        clusters = np.argmax(kernel_values, axis=1)

        # Calculate the new cluster centers
        new_centers = np.array([X[clusters == c].mean(axis=0) for c in range(k)])

        # Check for convergence
        if np.allclose(centers, new_centers):
            break

        centers = new_centers

    return clusters, centers


In [4]:
def one_hot_encoding(data):
    num_count = set()
    for i in data:
        num_count.add(i)
    num_count = list(num_count)
    data = np.copy(data)
    zeros = np.zeros((data.shape[0],len(num_count)))
    for i in range(data.shape[0]):
        zeros[i,num_count.index(data[i])] = 1
    return zeros

In [21]:
selected_index = next(sss.split(X,y))[1]
training_data = (X[selected_index])
label = np.argmax(one_hot_encoding(y),axis = 1)[selected_index]
pred = kmeans(training_data, 11, max_iterations=100, sigma=1)[0]

In [22]:
contigency_table = np.zeros((11,11),dtype = np.integer)
for i in range(len(pred)):
    contigency_table[label[i]][pred[i]] += 1
col_sum = np.sum(contigency_table,axis = 0)
row_sum = np.sum(contigency_table,axis = 1)
H_T = -np.sum(col_sum/np.sum(col_sum)*np.log(col_sum/np.sum(col_sum)))
H_C = -np.sum(row_sum/np.sum(row_sum)*np.log(row_sum/np.sum(row_sum)))
I_CT = 0
for i in range(11):
    for j in range(11):
        if (contigency_table[i][j] != 0):
            I_CT += (contigency_table[i][j]/np.sum(row_sum))*np.log((contigency_table[i][j]*np.sum(row_sum))/((row_sum[i])*(col_sum[j])))
NMI = I_CT/(np.sqrt(H_C*H_T))
NMI

c:\Users\Micha\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  """Entry point for launching an IPython kernel.


0.36427316061230425